In [0]:
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/Modulo de Seguimiento/')

import SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs,greatest
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

In [0]:
#Cuartiles
from pyspark.sql import functions as F

def categorize_by_quintiles(df, column_names, filter_condition, quintile_probabilities=[0.25, 0.5, 0.75, 1.0], relative_error=0.01):
    for column in column_names:
        # Filtrar y calcular los quintiles
        quintiles = df.filter(filter_condition).approxQuantile(column, quintile_probabilities, relative_error)
        Minimo = df.agg({column: "min"}).collect()[0][0]
        
        # Crear la nueva columna categorizada
        df = df.withColumn(
            f'{column}_Q',
            F.when(F.col(column) <= quintiles[0], f"1. {format(Minimo, '.3f')} - {format(quintiles[0], '.3f')}")
             .when(F.col(column) <= quintiles[1], f"2. {format(quintiles[0], '.3f')} - {format(quintiles[1], '.3f')}")
             .when(F.col(column) <= quintiles[2], f"3. {format(quintiles[1], '.3f')} - {format(quintiles[2], '.3f')}")
             .when(F.col(column) <= quintiles[3], f"4. {format(quintiles[2], '.3f')} - {format(quintiles[3], '.3f')}")
             .otherwise("98. Missing")
        )
    return df

In [0]:
#Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

In [0]:
from collections import Counter

def find_repeated_columns(df):
    column_names = df.columns
    column_counts = Counter(column_names)
    repeated_columns = [col for col, count in column_counts.items() if count > 1]
    return repeated_columns

In [0]:
def find_missing_columns(df1, df2):
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)
    
    missing_in_df2 = columns_df1 - columns_df2
    missing_in_df1 = columns_df2 - columns_df1
    
    print("Columnas en df1 pero no en df2:", missing_in_df2)
    print("Columnas en df2 pero no en df1:", missing_in_df1)

In [0]:
def generar_codmes(inicio, final):
    codmes_list = []
    anio_inicial = inicio // 100
    mes_inicial = inicio % 100
    
    anio_final = final // 100
    mes_final = final % 100
    
    anio = anio_inicial
    mes = mes_inicial
    
    # Generar la lista hasta el mes final
    while (anio < anio_final) or (anio == anio_final and mes <= mes_final):
        codmes_list.append(anio * 100 + mes)
        mes += 1
        if mes > 12:
            mes = 1  # Pasar a enero del siguiente año
            anio += 1
    return codmes_list

In [0]:
from pyspark.sql.functions import coalesce, lit

def dias_atraso(base, identificador, inicio, final): 
    df_list = []
    espacio = generar_codmes(inicio = inicio, final = final)
    for codmes in espacio:
        numeromes = (codmes // 100) * 12 + (codmes % 100)
        codmes_f1 = ((numeromes - 1 + 1) // 12) * 100 + ((numeromes + 11 - 11 * 1) % 12) + 1
        codmes_f2 = ((numeromes - 1 + 2) // 12) * 100 + ((numeromes + 11 - 11 * 2) % 12) + 1
        codmes_f3 = ((numeromes - 1 + 3) // 12) * 100 + ((numeromes + 11 - 11 * 3) % 12) + 1

        df_base = base.filter(f"CODMES = {codmes}").select("CODMES", identificador, "CTDMESMADURACION", "MTOSALDOCAPITALSOL","TIPBLOQUEOPRODUCTO", "CTDDIAATRASO")
        df_f1 = base.filter(f"CODMES = {codmes_f1}").select("CODMES", identificador, "CTDDIAATRASO")
        df_f2 = base.filter(f"CODMES = {codmes_f2}").select("CODMES", identificador, "CTDDIAATRASO")
        df_f3 = base.filter(f"CODMES = {codmes_f3}").select("CODMES", identificador, "CTDDIAATRASO")

        df_parcial = df_base.alias('A') \
                .join(df_f1.alias('B'), on = identificador, how = 'left') \
                .join(df_f2.alias('C'), on = identificador, how = 'left') \
                .join(df_f3.alias('D'), on = identificador, how = 'left') \
                .selectExpr('A.*', 
                            'COALESCE(B.CTDDIAATRASO, 0) AS CTDDIAATRASO_F1', 
                            'COALESCE(C.CTDDIAATRASO, 0) AS CTDDIAATRASO_F2', 
                            'COALESCE(D.CTDDIAATRASO, 0) AS CTDDIAATRASO_F3')
                
        df_list.append(df_parcial)
        print(f'Iteración {codmes}:', f't0:{codmes}', f't1:{codmes_f1}', f't2:{codmes_f2}', f't3:{codmes_f3}')

    df_final = df_list[0]
    for df in df_list[1:]:
        df_final = df_final.unionByName(df)

    return df_final

In [0]:
def rezago_codmes_col(base, rezago_mes):
    for i in range(0, rezago_mes + 1, 1):
        base = base.withColumn(f"CODMES_{i}", date_format(add_months(to_date(concat(col("codmes"), lit("01")), "yyyyMMdd"), -i), "yyyyMM"))
        base = base.withColumn(f"CODMES_{i}", col(f"CODMES_{i}").cast("integer"))
    return base

def avance_codmes_col(base, avance_mes):
    for l in range(0, avance_mes + 1, 1):
        base = base.withColumn(f"CODMES_F{l}", date_format(add_months(to_date(concat(col("codmes"), lit("01")), "yyyyMMdd"), l), "yyyyMM"))
        base = base.withColumn(f"CODMES_F{l}", col(f"CODMES_F{l}").cast("integer"))
    return base

In [0]:
hm_mtz_adm_hipotecario_driver = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_hm_mtz_admision_hipotecario_202503_F')

In [0]:
hm_adm_hip_trad_001 = hm_mtz_adm_hipotecario_driver.filter(
    (col("TIPO_FONDEO") == 'FONDEO BCP') &
    (col("CAMPANIA_AGRUPADA") == 'Tradicional') &
    (~col("DESTIPFINALIDADCREDHIPOTECARIO").isin(
        'AMPLIACION DE CREDITO PARA CONSTRUCCION',
        'AMPLIACION-REMOD.CASA CAMPO',
        'AMPLIACION-REMOD.CASA HABITACION',
        'Cambio de titularidad',
        'CAMBIO PLAZO',
        'CAMBIO VARIOS',
        'LIBRE DISP. CON GARANT.LOCAL COMERC.',
        'LIBRE DISPONIBILIDAD',
        'MIGRACION DE CREDITO',
        'MIGRACION DE MONEDA',
        'NUEVA AMPLIACION DE CREDITO',
        'Unificacion de credito hipotecario'
    ))
)

In [0]:
filtered_df = hm_adm_hip_trad_001.filter(col("codmes") >= 202201)

# Calculate the 99th percentile
percentiles_df = filtered_df.selectExpr("percentile_approx(MTOAPROBADO_SOLES, 0.99) as P_99")

# Show the result
percentiles_df.show()

In [0]:
hm_adm_hip_trad_001.createOrReplaceTempView("temp_hm_adm_hip_trad_001")

In [0]:
path_CSV = "abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/HIPOTECARIO_2025/APP/202503/HM_ADM_HIP_TRADPYME_2Q24"
 
spark.sql("""DROP TABLE IF EXISTS catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_ADM_HIP_TRADPYME_2Q24_202503;""")
 
spark.sql(f"""
    CREATE TABLE catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_ADM_HIP_TRADPYME_2Q24_202503
    USING CSV
    OPTIONS (header = "true", inferSchema = "true",delimiter = ";")    
    LOCATION '{path_CSV}'
""")

In [0]:
hm_adm_hip_trad_001_0 = spark.sql("""
SELECT 
    A.*, 
    B.PD_APP_HIP_TRDPYME_2Q24,
    B.XB_APP_HIP_TRDPYME_2Q24,
    B.XB_APP_HIP_TRDPYME_IDIO,
    B.XB_ADC_POND85_F AS XB_BASE, 
    B.FLAG_PYME,
    B.EXP_PCT_EVOL_SHIP_U6M_RT_U24 AS DEU_NOHIP_6_24, 
    B.EXP_PCT_EVOL_SHIP_U6M_RT_U24_A3,
    B.RCC_PCT_UTL6_UTL24_RT_U24 AS UTIL_PROM_6_24, 
    B.RCC_PCT_UTL6_UTL24_RT_U24_A2,
    B.SLD_PRM_PAS_MIN_24_24_RT_U24 AS PAS_MIN_PROM24, 
    B.SLD_PRM_PAS_MIN_24_24_RT_U24_I
FROM 
    temp_hm_adm_hip_trad_001 AS A
LEFT JOIN 
    (SELECT distinct a.* FROM catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_ADM_HIP_TRADPYME_2Q24_202503 as a WHERE CAMPANIA_AGRUPADA_NEW = 'Tradicional' and PD_APP_HIP_TRDPYME_2Q24 is not null) AS B 
ON 
    A.CODMES = B.CODMES AND A.CODCLAVECIC = B.CODCLAVECIC
""")


In [0]:
hm_adm_hip_trad_001_0.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = hm_adm_hip_trad_001_0.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
path_CSV = "abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/HIPOTECARIO_2025/APP/202503/MM_APPSCORE_HIP_TRD"
 
spark.sql("""DROP TABLE IF EXISTS catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_MM_APPSCORE_HIP_TRD_202503;""")
 
spark.sql(f"""
    CREATE TABLE catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_MM_APPSCORE_HIP_TRD_202503
    USING CSV
    OPTIONS (header = "true", inferSchema = "true",delimiter = ";")    
    LOCATION '{path_CSV}'
""")

In [0]:
hm_adm_hip_trad_001_0 = hm_adm_hip_trad_001_0.drop("EDAD")

In [0]:
hm_adm_hip_trad_001_0.createOrReplaceTempView("temp_hm_adm_hip_trad_001_0")

In [0]:
hm_adm_hip_trad_001_1=spark.sql("""
SELECT A.*, 
       B.XB_ADC_POND85_F, 
       B.XB_ADC_POND85,
       B.XB_AD_HOC,
       B.XB_ACT_S_ADCW,
       B.XB_PAS_S_ADCW, 
       B.XB_TRX_S_ADCW,
       B.XB_RCC_S_ADCW,
       B.XB_DEM_S_ADCW,
       B.ANT_LABORAL_MESES,
       B.ANT_LABORAL_MESES_E2,
       B.CAT_ZONA1,
       B.CAT_ZONA1_C_ST,
       B.CIIU_DEF,
       B.CL_CIIU_ST,
       B.EDAD,
       B.EDAD_E1,
       B.EST_CIV,
       B.EST_CIV_ST,
       B.ING_CASCADA3,
       B.ING_CASCADA3_D2,
       B.XB_HIP,
       B.LTV_1,
       B.LTV_1_D2,
       B.GRUPO_PLAZO,
       B.GRUPO_PLAZO_ST,
       B.RIESGO_ZONA_ST,
       B.RIESGO_ZONA_ST_ST,
       B.XB_ADC_POND85_2,
       B.XB_AD_HOC_2,
       B.XB_DEM_S_ADCW_2,
       B.FLG_SOLTERO_C AS FLG_SOLTERO_ESPJ,
       B.SEGMENTO AS SEGMENTO_ESPJ,
       B.SEGMENTO2_st,
       B.FLG_NODEP AS FLG_NODEP_ESPJ,
       B.ZONA_PAUTA5,
       B.ZONA_PAUTA5_st,
       B.D_PYME,
       CASE 
           WHEN B.ZONA_PAUTA5 IS NULL THEN '99. Missing'
           WHEN B.ZONA_PAUTA5 = 'LIMA TOP' THEN '1. Lima Top'
           WHEN B.ZONA_PAUTA5 = 'LIMA MODERNA' THEN '2. Lima Moderna'
           WHEN B.ZONA_PAUTA5 = 'PROVINCIA TOP' THEN '3. Provincia Top'
           WHEN B.ZONA_PAUTA5 = 'OTROS' THEN '4. Otros'
           ELSE '98. N.A.' 
       END AS ZONA_PAUTA5_F,
       B.PD_P7_ajus AS PD_BASE_ESPEJO,
       B.SC_P7_ajus AS SC_BASE_ESPEJO,
       B.XB_P7_ajus AS XB_BASE_ESPEJO
FROM temp_hm_adm_hip_trad_001_0 A
LEFT JOIN catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_MM_APPSCORE_HIP_TRD_202503 B ON A.CODSOLICITUD = B.CODSOLICITUD
""")


In [0]:
hm_adm_hip_trad_001_1.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = hm_adm_hip_trad_001_1.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
PD_SEG='PD_APP_HIP_TRDPYME_2Q24'
SCORE='SC_APP_TRAD'
PAUTA=335

In [0]:
hm_adm_hip_trad_002 = hm_adm_hip_trad_001_1.withColumn("RANGO_SCORE", when(col(SCORE).isNull(), '99. Missing')
                   .when(col(SCORE) < 335, '01. <0 - 335>')
                   .when(col(SCORE) < 365, '02. [335 - 365>')
                   .when(col(SCORE) < 415, '03. [365 - 415>')
                   .when(col(SCORE) < 485, '04. [415 - 485>')
                   .when(col(SCORE) >= 485, '05. [485 - ++>')
                   .otherwise('98. Otro'))

In [0]:
# Add the RANGO_PD column
hm_adm_hip_trad_002 = hm_adm_hip_trad_002.withColumn("RANGO_PD", when(col(PD_SEG).isNull(), '99. Missing')
                   .when(col(PD_SEG) <= 0.0045641, '01. [0 - 0.45%]')
                   .when(col(PD_SEG) <= 0.0151879, '02. <0.45 - 1.51%]')
                   .when(col(PD_SEG) <= 0.0353823, '03. <1.51 - 3.53%]')
                   .when(col(PD_SEG) <= 0.0581041, '04. <3.53 - 5.81%]')
                   .when(col(PD_SEG) > 0.0581041, '05. <5.81 - 100%]')
                   .otherwise('98. Otro'))

In [0]:
# Add the MONTO_R column
hm_adm_hip_trad_002 = hm_adm_hip_trad_002.withColumn("MONTO_R", when(col("MTOAPROBADO_SOLES").isNull(), '99. Missing')
                   .when(col("MTOAPROBADO_SOLES") <= 237500, '1. [0 - 237,500]')
                   .when(col("MTOAPROBADO_SOLES") <= 340000, '2. <237,500 - 340,000]')
                   .when(col("MTOAPROBADO_SOLES") <= 460000, '3. <340,000 - 460,000]')
                   .when(col("MTOAPROBADO_SOLES") <= 680000, '4. <460,000 - 680,000]')
                   .otherwise('5. <680,000 - ++]'))

# Add the PAUTA column
hm_adm_hip_trad_002 = hm_adm_hip_trad_002.withColumn("PAUTA", when(col(SCORE) >= PAUTA , 1).otherwise(0))

# Add the MISS_PD column
hm_adm_hip_trad_002 = hm_adm_hip_trad_002.withColumn("MISS_PD", when(col(PD_SEG).isNull(), 1).otherwise(0))

# Add the MISS_MONTO column
hm_adm_hip_trad_002 = hm_adm_hip_trad_002.withColumn("MISS_MONTO", when(col("MTOAPROBADO_SOLES").isNull(), 1).otherwise(0))

# Add the FLG_ALTOS column
hm_adm_hip_trad_002 = hm_adm_hip_trad_002.withColumn("FLG_ALTOS", when(col("MTOAPROBADO_SOLES") >= 2166210, 1).otherwise(0))

# Add the DEU_NOHIP_6_24_F column
hm_adm_hip_trad_002 = hm_adm_hip_trad_002.withColumn("DEU_NOHIP_6_24_F", when(col("DEU_NOHIP_6_24").isin(
    None, 1111111111, -1111111111, 2222222222, -2222222222, 3333333333, -3333333333, 
    4444444444, 5555555555, 6666666666, 7777777777, 99999), None).otherwise(col("DEU_NOHIP_6_24")))

# Add the UTIL_PROM_6_24_F column
hm_adm_hip_trad_002 = hm_adm_hip_trad_002.withColumn("UTIL_PROM_6_24_F", when(col("UTIL_PROM_6_24").isin(
    None, 1111111111, -1111111111, 2222222222, -2222222222, 3333333333, -3333333333, 
    4444444444, 5555555555, 6666666666, 7777777777, 99999), None).otherwise(col("UTIL_PROM_6_24")))

# Add the PAS_MIN_PROM24_F column
hm_adm_hip_trad = hm_adm_hip_trad_002.withColumn("PAS_MIN_PROM24_F", when(col("PAS_MIN_PROM24").isin(
    None, 1111111111, -1111111111, 2222222222, -2222222222, 3333333333, -3333333333, 
    4444444444, 5555555555, 6666666666, 7777777777, 99999), None).otherwise(col("PAS_MIN_PROM24")))

In [0]:
hm_adm_hip_trad = hm_adm_hip_trad.withColumn("GRUPO_PLAZO", 
    when(col("GRUPO_PLAZO") == 1, lit("1. Menor a 4 años"))
    .when(col("GRUPO_PLAZO") == 2, lit("2. De 5 a 9 años"))
    .when(col("GRUPO_PLAZO") == 3, lit("3. De 10 a 14 años"))
    .when(col("GRUPO_PLAZO") == 4, lit("4. De 15 a 19 años"))
    .when(col("GRUPO_PLAZO") == 5, lit("5. De 20 a 24 años"))
    .when(col("GRUPO_PLAZO") == 6, lit("6. De 25 a 29 años"))
    .when(col("GRUPO_PLAZO") == 7, lit("7. De 30 a 34 años"))
    .when(col("GRUPO_PLAZO") == 8, lit("8. 35 años o más"))
)


In [0]:
sorted(hm_adm_hip_trad.columns)

In [0]:
hm_adm_hip_trad.createOrReplaceTempView("temp_view")
spark.sql("CREATE OR REPLACE TABLE catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_ADM_HIP_TRAD_202503_F LOCATION 'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/HIPOTECARIO_2025/APP/202503/T45988_HM_ADM_HIP_TRAD_202503_F' AS SELECT * FROM temp_view")

In [0]:
hm_adm_hip_trad.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = hm_adm_hip_trad.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)